<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
# import os; os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# except:
#   # Invalid device or cannot modify virtual devices once initialized.
#   pass

2023-04-23 18:53:07.649737: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-23 18:53:07.701338: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 18:53:09.670200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import jax
from typing import Any, Callable, Sequence, Union
from jax import lax, random, numpy as jnp
from flax.core import freeze, unfreeze
from flax import linen as nn
import optax

from iqadatasets.datasets import *

## Load the data

> We're going to employ `iqadatasets` to ease the loading of the data.

In [ ]:
dst_train = TID2008("/media/disk/databases/BBDD_video_image/Image_Quality/TID/TID2008/")
dst_val = TID2013("/media/disk/databases/BBDD_video_image/Image_Quality/TID/TID2013/")

In [ ]:
img, img_dist, mos = next(iter(dst_train.dataset))
img.shape, img_dist.shape, mos.shape

2023-04-23 18:53:14.371382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

In [ ]:
img, img_dist, mos = next(iter(dst_val.dataset))
img.shape, img_dist.shape, mos.shape

2023-04-23 18:55:37.101850: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

## Define the model

> We're going to define a simple PerceptNet without GDN.

In [ ]:
class PerceptNet(nn.Module):
    """IQA model inspired by the visual system."""

    @nn.compact
    def __call__(self,
                 inputs,
                 ):
        outputs = nn.Conv(features=3, kernel_size=(5,5), strides=1, padding="SAME")(inputs)
        outputs = nn.relu(outputs)
        outputs = nn.max_pool(outputs, window_shape=(2,2), strides=(2,2))
        outputs = nn.Conv(features=6, kernel_size=(5,5), strides=1, padding="SAME")(inputs)
        outputs = nn.relu(outputs)
        outputs = nn.max_pool(outputs, window_shape=(2,2), strides=(2,2))
        outputs = nn.Conv(features=128, kernel_size=(5,5), strides=1, padding="SAME")(inputs)
        outputs = nn.relu(outputs)
        return outputs

In [ ]:
key1, key2 = random.split(random.PRNGKey(0), 2)
inputs = random.uniform(key2, shape=(384,512,3))
model = PerceptNet()
params = model.init(key1, inputs)
jax.tree_util.tree_map(lambda x: x.shape, params)

FrozenDict({
    params: {
        Conv_0: {
            bias: (3,),
            kernel: (5, 5, 3, 3),
        },
        Conv_1: {
            bias: (6,),
            kernel: (5, 5, 3, 6),
        },
        Conv_2: {
            bias: (128,),
            kernel: (5, 5, 3, 128),
        },
    },
})

## Define the training loop

In [ ]:
from functools import partial

@partial(jax.jit, static_argnums=(0, 1))
def update_step(apply_fn, tx, inputs, labels, opt_state, params, state):
    def loss(params):
        ## Forward pass through the model
        img_pred = apply_fn(params, 
                            img)
        img_dist_pred = apply_fn(params, 
                                 img_dist)
        ## Calculate the distance
        dist = ((img_pred - img_dist_pred)**2).sum(axis=(1,2,3))**(1/2)
        
        ## Calculate pearson correlation
        y_true = dist.squeeze()
        y_pred = mos.squeeze()
        y_true_mean = y_true.mean()
        y_pred_mean = y_pred.mean()
        num = y_true-y_true_mean
        num *= y_pred-y_pred_mean
        num = num.sum()
        denom = jnp.sqrt(jnp.sum((y_true-y_true_mean)**2))
        denom *= jnp.sqrt(jnp.sum((y_pred-y_pred_mean)**2))
        return num/denom
    (l, updated_state), grads = jax.value_and_grad(loss, has_aux=True)(params)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return opt_state, params, updated_state, l

## Define the evaluation loop

In [ ]:
from functools import partial

@partial(jax.jit, static_argnums=(0))
def eval_step(apply_fn, img, img_dist, mos, params):
    def loss(params):
        ## Forward pass through the model
        img_pred = apply_fn(params, 
                            img)
        img_dist_pred = apply_fn(params, 
                                 img_dist)
        ## Calculate the distance
        dist = ((img_pred - img_dist_pred)**2).sum(axis=(1,2,3))**(1/2)
        
        ## Calculate pearson correlation
        y_true = dist.squeeze()
        y_pred = mos.squeeze()
        y_true_mean = y_true.mean()
        y_pred_mean = y_pred.mean()
        num = y_true-y_true_mean
        num *= y_pred-y_pred_mean
        num = num.sum()
        denom = jnp.sqrt(jnp.sum((y_true-y_true_mean)**2))
        denom *= jnp.sqrt(jnp.sum((y_pred-y_pred_mean)**2))
        return num/denom
    
    l = loss(params)
    return l

## Train the model

In [ ]:
key1, key2 = random.split(random.PRNGKey(0), 2)
inputs = random.uniform(key2, shape=(384,512,3))
model = PerceptNet()
params = model.init(key1, inputs)
tx = optax.adam(learning_rate=3e-4)
opt_state = tx.init(params)

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32
losses, losses_val = [], []
for epoch in range(EPOCHS):
    
    ## Training
    loss_epoch = []
    for img, img_dist, mos in dst_train.dataset.batch(BATCH_SIZE):
        img, img_dist, mos = img.numpy(), img_dist.numpy(), mos.numpy()
        opt_state, params, loss = update_step(
        model.apply, tx, img, img_dist, mos, opt_state, params)
        loss_epoch.append(loss)
        # break
    losses.append(jnp.mean(jnp.array(loss_epoch)))
    
    ## Evaluation
    loss_epoch_val = []
    for img, img_dist, mos in dst_val.dataset.batch(BATCH_SIZE):
        img, img_dist, mos = img.numpy(), img_dist.numpy(), mos.numpy()
        loss_val = eval_step(model.apply, img, img_dist, mos, params)
        loss_epoch_val.append(loss_val)
        # break
    losses_val.append(jnp.mean(jnp.array(loss_epoch_val)))

    ## Logging
    print(f"Epoch {epoch} -> Loss (Train): {losses[-1]} | Loss (Validation): {losses_val[-1]}")
    # break

2023-04-23 19:01:10.347231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-23 19:01:28.015166: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 0 -> Loss (Train): -0.557887613773346 | Loss (Validation): -0.7108475565910339


2023-04-23 19:03:05.010286: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-23 19:03:24.610020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1 -> Loss (Train): -0.6508658528327942 | Loss (Validation): -0.7451121211051941


2023-04-23 19:03:56.007672: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-23 19:04:13.087976: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 2 -> Loss (Train): -0.6950182914733887 | Loss (Validation): -0.76376873254776


2023-04-23 19:04:40.923255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-23 19:04:57.161878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 3 -> Loss (Train): -0.7255797386169434 | Loss (Validation): -0.7783637642860413


2023-04-23 19:05:25.436143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-23 19:05:41.914958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 4 -> Loss (Train): -0.7484660744667053 | Loss (Validation): -0.7901502251625061
CPU times: user 13min 23s, sys: 19min 18s, total: 32min 42s
Wall time: 4min 59s
